In [1]:
from fastapi.middleware.cors import CORSMiddleware
from fastapi import FastAPI, Request
import requests

%run ../knn/kitsuknn.ipynb


In [ ]:
import uvicorn
import nest_asyncio
from pydantic import BaseModel
import pandas as pd
from fastapi.encoders import jsonable_encoder

app = FastAPI()


nest_asyncio.apply()



class Base(BaseModel):
    id: int

# Searches for the anime's category, calls the model
@app.post("/recommend")
def recommend(body: Base):

    r = requests.get(f'https://kitsu.io/api/edge/anime/{body.id}/categories')
    data = r.json()
    categories = []
    for category in data["data"][:3]:
        categories.append({'slug' : category["attributes"]["slug"],
            'title' :category["attributes"]["title"]})
        
    animes = []

    print(categories[0]["slug"])

    
    animess_0 = get_all_animes_per_categorie(categories[0]["slug"])
    # animess_1 = get_all_animes_per_categorie(categories[1]["slug"])
    # animess_2 = get_all_animes_per_categorie(categories[2]["slug"])
    animes = animes + animess_0 

    df = create_df(animes)

    results = knn_model(df)


    # results['animes'] = [row.to_dict() for row in results['animes']]

    print('RESULTS ANIMES',results)

        
    return results

if __name__ == "__main__":

    app.add_middleware(
        CORSMiddleware,
        allow_origins=["*"],  
        allow_credentials=True,
        allow_methods=["*"],
        allow_headers=["*"], 
    )

    
    uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [19576]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:50107 - "OPTIONS /recommend HTTP/1.1" 200 OK
bounty-hunter
Found the amount of animes 33
RESULTS ANIMES [['1', '2', '7726', '11170', '16', '44226', '2180', '7722', '130', '1060', '198', '967', '4188', '3662', '4330', '388', '1317', '49', '396', '1829'], [5.88418203051333e-15, 0.5112030799146194, 0.920113529075189, 0.9327378995421041, 0.9337713970280841, 0.9359120988383216, 0.9362092230508692, 0.9368324018923898, 0.9370066392775508, 0.9370329827573092, 0.9380337113336495, 0.9385695717690746, 0.9587554281200329, 0.9592399425661382, 0.9596656747567988, 0.9601756133092685, 0.9643068538695441, 0.9647040257463092, 0.9655017137293509, 0.9657755484887341]]
INFO:     127.0.0.1:50107 - "POST /recommend HTTP/1.1" 200 OK
